# Bioinformatika Lab. 1

In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from math import ceil
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Orf import Orf
from Bio.Alphabet import generic_dna, generic_rna

In [2]:
rec = SeqIO.read("plazmide.fasta", "fasta")
sequence = rec.seq

In [3]:
print(rec)

ID: plazmide
Name: plazmide
Description: plazmide
Number of features: 0
Seq('TTGCGCCGGCGAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATC...CCT', SingleLetterAlphabet())


In [4]:
start = ["ATG"]
stop = ["TAA", "TAG", "TGA"]
convert = {"C" : "G", "G" : "C", "T" : "A", "A" : "T"}

In [5]:
def complement(data):
    compl = []
    for var in data:
        compl.append(convert[var])
    return compl

In [6]:
def reverse(data):
    return "".join(list(reversed(data)))

In [7]:
def findOrfs(dna, st, numb, revert=False):
    frame = "-" if revert else ""
    frame += str(st)
    orfstart = -1
    orfend = -1
    orfs = []
    data = reverse(complement(dna)) if revert else dna
    for index in range(st, len(data), 3):
        codon = data[index:index+3]
        last_start = 0
        if codon in start:
            #print("Start: {} at {}".format(codon, index))
            if orfstart == -1:
                orfstart = index
        if codon in stop:
            #print("Stop: {} at {}".format(codon, index))
            orfend = index + 3
            if orfend > orfstart and orfstart != -1:
                value = data[orfstart:orfend]
                if orfend - orfstart > 100:
                    orfs.append(Orf(orfstart, orfend, frame, value, (orfend - orfstart), len(orfs)+numb+1))
                orfstart = -1
    return orfs

In [8]:
def getAllOrfs(dna):
    orfs = []
    orfs.extend(findOrfs(dna, 0, 0))
    numb = len(orfs)
    orfs.extend(findOrfs(dna, 1, numb))
    numb = len(orfs)
    orfs.extend(findOrfs(dna, 2, numb))
    numb = len(orfs)
    orfs.extend(findOrfs(dna, 0, numb, True))
    numb = len(orfs)
    orfs.extend(findOrfs(dna, 1, numb, True))
    numb = len(orfs)
    orfs.extend(findOrfs(dna, 2, numb, True))
    return orfs    

In [9]:
orfs = getAllOrfs(sequence) # mind that indexes are wrong for reversed sequences
orfs.sort(key=lambda x: x.name, reverse=False)
print("Number of orfs: {}".format(len(orfs)))
for o in orfs:
    print(o)

Number of orfs: 29
1 <Start: 1494, Stop: 1812, Length: 318, Frame: 0>
2 <Start: 2400, Stop: 2793, Length: 393, Frame: 0>
3 <Start: 2799, Stop: 2925, Length: 126, Frame: 0>
4 <Start: 3279, Stop: 3462, Length: 183, Frame: 0>
5 <Start: 4404, Stop: 4671, Length: 267, Frame: 0>
6 <Start: 4677, Stop: 4848, Length: 171, Frame: 0>
7 <Start: 4857, Stop: 5007, Length: 150, Frame: 0>
8 <Start: 391, Stop: 1264, Length: 873, Frame: 1>
9 <Start: 1684, Stop: 2644, Length: 960, Frame: 1>
10 <Start: 2863, Stop: 3088, Length: 225, Frame: 1>
11 <Start: 533, Stop: 962, Length: 429, Frame: 2>
12 <Start: 1946, Stop: 2117, Length: 171, Frame: 2>
13 <Start: 2261, Stop: 2372, Length: 111, Frame: 2>
14 <Start: 2990, Stop: 3200, Length: 210, Frame: 2>
15 <Start: 291, Stop: 393, Length: 102, Frame: -0>
16 <Start: 4092, Stop: 4305, Length: 213, Frame: -0>
17 <Start: 4962, Stop: 5112, Length: 150, Frame: -0>
18 <Start: 5169, Stop: 5280, Length: 111, Frame: -0>
19 <Start: 1618, Stop: 1738, Length: 120, Frame: -1>
20

In [10]:
def toRNAs(orfs):
    rnas = []
    for orf in orfs:
        rnas.append(toRNA(orf.value))
    return rnas

In [11]:
def toRNA(dna):
    conv = {"T" : "A", "A" : "U", "C" : "G", "G" : "C"}
    rna = ""
    for i in dna:
        rna += str(conv[i])
    return rna

In [12]:
def toProteins(rnas):
    proteins = []
    for rna in rnas:
        r = Seq(rna, generic_rna)
        proteins.append(r.translate())
    return proteins

In [13]:
rnas = toRNAs(orfs)

In [14]:
proteins = toProteins(rnas)

### Alignment

In [15]:
from Bio import Align
import numpy as np
aligner = Align.PairwiseAligner()
aligner.open_gap_score = -10
aligner.extend_gap_score = -0.5

In [16]:
class Comparison:
    def __init__(self, firstOrf, secondOrf, alignment):
        self.firstOrf = firstOrf
        self.secondOrf = secondOrf
        self.alignment = alignment
    def __str__(self):
        return 'Comparison <Score: {}>'.format(self.alignment.score)

In [17]:
comparisons = []
arr = np.zeros((len(orfs), len(orfs)))
for i in range(0, len(orfs)):
    for j in range(0, len(orfs)):
        arr[i,j] = aligner.score(orfs[i].value, orfs[j].value)
        #al = aligner.align(orfs[i].value, orfs[j].value)
        #comparisons.append(Comparison(orfs[i],orfs[j],al))

In [18]:
comparisons.sort(key=lambda x: x.alignment.score, reverse=False)

In [26]:
for c in comparisons:
    print("{} and {} Score: {}".format(c.firstOrf.name, c.secondOrf.name, c.alignment.score))

In [30]:
print(arr.shape)

(29, 29)


In [31]:
import pandas as pd
from scipy.spatial import distance_matrix

data = np.asarray(arr)
ctys = []
for i in range(0, len(orfs)):
    ctys.append(str(i))
df = pd.DataFrame(data, columns=ctys, index=ctys)

In [34]:
distance = pd.DataFrame(distance_matrix(df.values, df.values), index=df.index, columns=df.index)

In [37]:
print(distance)

              0            1            2            3            4  \
0      0.000000   483.161981   673.958641   556.491015   383.290360   
1    483.161981     0.000000   876.489447   771.723234   578.815169   
2    673.958641   876.489447     0.000000   291.619615   526.235451   
3    556.491015   771.723234   291.619615     0.000000   401.015897   
4    383.290360   578.815169   526.235451   401.015897     0.000000   
5    585.327472   796.154665   251.826329   183.749830   432.956984   
6    617.887328   827.010731   189.558698   236.716075   464.951342   
7   1861.845523  1687.014597  2192.612597  2139.438595  1971.405907   
8   2054.976460  1903.424742  2358.283910  2318.622436  2162.550520   
9    457.751843   679.433404   410.434526   276.197393   322.961685   
10   669.250887   541.068157  1061.888177   960.072133   770.595387   
11   566.413056   779.206167   278.986559   231.401167   425.203775   
12   683.613195   881.547219   166.606572   315.160673   541.461448   
13   4

In [39]:
np.savetxt(r'/media/rytisk/01D31DACBEB500B0/Projects/bioinformatics/phylip.txt', distance.values, fmt='%f')

#### Maybe: https://biopython.org/wiki/Phylo#Tree_Construction